In [18]:
import pandas as pd
import numpy as np

# =========================
# 1) Load raw data
# =========================
df = pd.read_csv("../data/raw/samples.csv")

# =========================
# 2) Parse / clean basic fields
# =========================
# Normalize known bad readings -> NaN
bad_tokens = ["error", "bad_reading", "N/A"]
df["value_clean"] = df["value"].replace(bad_tokens, np.nan)

# Convert to numeric (invalid -> NaN)
df["value_num"] = pd.to_numeric(df["value_clean"], errors="coerce")

# Parse date
df["date_dt"] = pd.to_datetime(df["date"], format="%Y-%m-%d", errors="coerce")

# =========================
# 3) Unit validation
# =========================
unit_rules = {
    "Water": "mg/kg",
    "Sulfur": "mg/kg",
    "Chloride": "mg/kg",
    "Ash": "% m/m",
    "Viscosity": "cSt",
}

df["expected_unit"] = df["parameter"].map(unit_rules)

# unit_ok is True only when we have a rule AND units match
df["unit_ok"] = (df["expected_unit"].notna()) & (df["unit"] == df["expected_unit"])

# =========================
# 4) Analysis-ready dataset
# =========================
df_analysis = df[(df["unit_ok"] == True) & (df["value_num"].notna())].copy()

print("Raw shape:", df.shape)
print("Analysis shape (unit_ok & numeric):", df_analysis.shape)

# =========================
# 5) IQR outliers per parameter
# =========================
def detect_iqr_outliers(group: pd.DataFrame) -> pd.DataFrame:
    """
    Detect outliers using IQR rule within a single parameter group.
    Returns only outlier rows, enriched with bounds + iqr + parameter.
    Edge-case: if IQR == 0 (flat distribution), returns empty outliers for this group.
    """
    q1 = group["value_num"].quantile(0.25)
    q3 = group["value_num"].quantile(0.75)
    iqr = q3 - q1

    # Guard: no spread -> no IQR-based outliers
    if pd.isna(iqr) or iqr == 0:
        return group.iloc[0:0].copy()

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    out = group[(group["value_num"] < lower) | (group["value_num"] > upper)].copy()
    out["iqr_lower"] = lower
    out["iqr_upper"] = upper
    out["iqr"] = iqr
    out["parameter"] = group.name  # ensure parameter is always a column

    return out

outliers = (
    df_analysis
    .groupby("parameter", group_keys=False)
    .apply(detect_iqr_outliers)
    .reset_index(drop=True)
)

print("Outliers shape:", outliers.shape)

# =========================
# 6) Severity scoring (distance beyond bound / IQR)
# =========================
if not outliers.empty:
    outliers["severity"] = np.where(
        outliers["value_num"] > outliers["iqr_upper"],
        (outliers["value_num"] - outliers["iqr_upper"]) / outliers["iqr"],
        (outliers["iqr_lower"] - outliers["value_num"]) / outliers["iqr"],
    )

    outliers["severity_level"] = pd.cut(
        outliers["severity"],
        bins=[-np.inf, 1, 3, np.inf],
        labels=["low", "medium", "high"],
    )
else:
    # Keep consistent columns even if no outliers
    outliers["severity"] = pd.Series(dtype="float64")
    outliers["severity_level"] = pd.Series(dtype="category")

# =========================
# 7) Final alerts table
# =========================
alerts_outliers = outliers[[
    "sample_id", "product", "parameter", "value_num", "unit", "date_dt",
    "iqr_lower", "iqr_upper", "severity", "severity_level"
]].sort_values(["severity", "date_dt"], ascending=[False, False])

# Quick views
display(alerts_outliers.head(10))

print("Outliers per parameter:")
print(outliers["parameter"].value_counts())

outliers_summary = (
    outliers
    .groupby("parameter")
    .agg(
        outlier_count=("sample_id", "count"),
        min_outlier=("value_num", "min"),
        max_outlier=("value_num", "max"),
        max_severity=("severity", "max"),
    )
    .sort_values("outlier_count", ascending=False)
)

display(outliers_summary)

# =========================
# 8) Save artifacts
# =========================
outliers.to_csv("../data/processed/outliers_iqr.csv", index=False)
alerts_outliers.to_csv("../data/processed/alerts_outliers_iqr.csv", index=False)

# Optional sanity check: water outliers
display(outliers[outliers["parameter"] == "Water"][["sample_id", "product", "value_num", "unit", "date_dt"]])


Raw shape: (120, 11)
Analysis shape (unit_ok & numeric): (109, 11)
Outliers shape: (4, 14)


,sample_id,product,parameter,value_num,unit,date_dt,iqr_lower,iqr_upper,severity,severity_level
0,1,Diesel,Ash,0.36,% m/m,2026-01-07,-0.0025,0.0649,17.513353,high
2,94,HSFO,Water,825.00,mg/kg,2026-01-16,10.4150,225.8550,11.124118,high
3,80,HSFO,Water,825.00,mg/kg,2026-01-13,10.4150,225.8550,11.124118,high
1,58,Diesel,Viscosity,810.00,cSt,2026-01-26,34.1000,362.1000,5.462195,high


Outliers per parameter:
parameter
Water        2
Ash          1
Viscosity    1
Name: count, dtype: int64


,outlier_count,min_outlier,max_outlier,max_severity
parameter,,,,
Water,2,825.00,825.00,11.124118
Ash,1,0.36,0.36,17.513353
Viscosity,1,810.00,810.00,5.462195


,sample_id,product,value_num,unit,date_dt
2,94,HSFO,825.0,mg/kg,2026-01-16
3,80,HSFO,825.0,mg/kg,2026-01-13
